In [ ]:
#%pip install httpx ddgs numexpr --upgrade

In [35]:
import re
import traceback
from ddgs import DDGS
import httpx
import numexpr as ne
import wikipedia
from pprint import PrettyPrinter
from src.simpleChatBot_chatgpt import SimpleChatBot as ChatBot

# Configure pprint with line wrapping at 100 characters
pp = PrettyPrinter(width=100)
pprint = pp.pprint

In [21]:
prompt="""
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.

**IMPORTANT**: 
- Use Observation as the primary source for your answer.
- Always consult Observation for context, facts and source of information.

Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

*Your available actions are:*
1. search:
e.g. search: <search query>
Searches the internet for information related to the query. Use this for general information retrieval.

2. wikisearch:
e.g. wikisearch: <search query>
Searches Wikipedia for the same query used in search. Use this for Wikipedia-specific information.

3. calculate:
e.g. calculate: <mathematical expression>
Runs a calculation and returns the number - uses Python, so be sure to use floating-point syntax if necessary.

4. halt:
e.g. halt: <question>
End the coversation.

5. count:
e.g. count: {'word'='hello', 'letter'='l'}
Runs count function which returns the count of a letter in a word - e.g. count({'word'='help', 'letter'='h'}) returns "There are 1 Hs in help".

# Error Handling:
If an action fails or returns no relevant information, use Thought to describe the issue and choose an alternative action. For example:
- If `search` returns no results, use `wikisearch`.
- If a calculation is invalid, use Thought to explain the error and try a different approach.

# Example Session:
Question: What year was the Eiffel Tower built?
Thought: I need to find the construction year of the Eiffel Tower. I can search the internet for this.
Action: search: construction year of Eiffel Tower
PAUSE

*While pausing, you will be called again with this:*

Observation: The Eiffel Tower was built in 1889.

*You then output:*

Answer: The Eiffel Tower was built in 1889.

# Additional Notes:
- Ensure the Answer strictly addresses the question asked, unless explicitly instructed otherwise.
- If the question requires multiple steps, use Thought to summarize previous steps and maintain context.
""".strip()

In [48]:
# This code is adapted from Simon Willison https://til.simonwillison.net/llms/python-react-pattern

ACTION_RE=re.compile(r"^Action:\s*([\w]+)\s*:\s*(.+)$", re.DOTALL)


def query(question, max_turns=5):
    i=0
    bot=ChatBot(prompt)
    next_prompt=question
    label="BOT RESPONSE:"
    while i<max_turns:
        i+=1
        try:
            result=bot(next_prompt)
            print(f"{label} {result}")

            # Extract actions from result
            actions=[ACTION_RE.match(action.strip()) for action in result.split("\n") if
                     ACTION_RE.match(action.strip())]

            if actions:
                action, action_input=actions[0].groups()
                action_input=action_input.strip()

                if action not in known_actions:
                    raise ValueError(f"Unknown action: {action}: {action_input}")

                pprint(f" -- Running {action} {action_input}")
                observation=known_actions[action](action_input)
                print(f"Observation: {observation}")

                next_prompt=f"Observation: {observation}"
            else:
                # print("Bot History", bot.messages)
                return  # No action found, exit loop

        except Exception as e:
            print(f"Error: {e}")
            traceback.print_exc()
            return

def websearch(q, max_results=1):
    try:
        results=DDGS().text(q, max_results=max_results)
        # results[0]["body"] = "I don't know the answer, I am sorry."
        # print(f"WEB SEARCH RESULTS>>> {results}")
        if results:
            return results[0]["body"]
        else:
            return "No relevant information found."
    except Exception as e:
        print(f"Search error: {e}")
        return "Error: Could not retrieve search results."


def wikisearch(q):
    """Search Wikipedia for information using the wikipedia package."""
    try:
        result = wikipedia.summary(q, sentences=3)
        return result
    except wikipedia.exceptions.DisambiguationError as e:
        # Return first few disambiguation options
        options = e.options[:3]
        return f"Multiple results found. Did you mean: {', '.join(options)}?"
    except wikipedia.exceptions.PageError:
        return f"No Wikipedia page found for '{q}'."
    except Exception as e:
        return f"Error: {str(e)}"


def calculate(expression):
    try:
        result=ne.evaluate(expression)
        print(f"CALCULATED>>> {result}")
        return result
    except Exception as e:  #Catches the broader set of Numexpr errors
        print(f"Calculation error: {e}")
        return "Error: Invalid calculation expression."


def halt(q):
    return "Nothing else can be done at this time."

import ast
def count_letters(inputs:dict):
    # print ("INPUTS>>>", inputs)
    if isinstance(inputs, str):
        inputs = ast.literal_eval(inputs)
    word = inputs.get("word")
    letter = inputs.get("letter")
    ans = word.lower().count(letter.lower()) if word and letter else 0
    return f"There are {ans} {letter.upper()}s in {word}"

def count_letters1(word:str, letter:str):
    return word.lower().count(letter.lower()) if word and letter else 0

known_actions={
    "search": websearch,
    "wikisearch": wikisearch,
    "calculate": calculate,
    "halt": halt,
    "count": count_letters,
}

In [32]:
count_letters("{'word':'Raspberry', 'letter':'R'}")

'There are 3 Rs in Raspberry'

In [37]:
query("How many Rs are in 'Raspberry'?")

("BOT RESPONSE: Thought: I need to count the number of letter 'R' in the word 'Raspberry'. I can "
 'use the count function for this. \n'
 "Action: count: {'word': 'Raspberry', 'letter': 'R'} \n"
 'PAUSE')
" -- Running count {'word': 'Raspberry', 'letter': 'R'}"
'Observation: There are 3 Rs in Raspberry'
"BOT RESPONSE: Answer: There are 3 Rs in 'Raspberry'."


In [38]:
query("My name is Happy!")

('BOT RESPONSE: Thought: I see that you\'ve introduced yourself as "Happy." This is a friendly '
 "greeting, but I need to determine if there's a specific question or topic you'd like to "
 'discuss. \n'
 '\n'
 'Action: halt: What would you like to talk about, Happy? \n'
 'PAUSE')
' -- Running halt What would you like to talk about, Happy?'
'Observation: Nothing else can be done at this time.'
("BOT RESPONSE: Answer: It seems there isn't a specific question or topic to discuss at this "
 'moment, Happy. If you have something in mind, feel free to share!')


In [50]:
query("What is the capital of England?")

BOT RESPONSE: Thought: I need to find out the capital city of England. This is a straightforward fact that I can likely find quickly. I will search the internet for it.  
Action: search: capital of England  
PAUSE
' -- Running search capital of England'
Observation: London is the capital and largest city of both England and the United Kingdom, with a population of 9.1 million people in 2024. Its wider metropolitan area is the largest in Western Europe, with a population of 15.1 million. London stands on the River Thames in southeast England, at the head of a 50-mile (80 km) tidal estuary down to the North Sea, and has been a major settlement for nearly 2,000 years. Its ancient core and financial centre, the City of London, was founded by the Romans as Londinium and has retained its medieval boundaries. The City of Westminster, to the west of the City of London, has been the centuries-long host of the national government and parliament. London grew rapidly in the 19th century, becoming 

In [8]:
query("What is (2+4) times a number?")

BOT RESPONSE: Thought: The question asks for the result of (2+4) multiplied by an unknown number.  I need to get the user to provide the unknown number.

Action: ask: Please provide the number you want to multiply (2+4) by.

PAUSE

 -- Running ask Please provide the number you want to multiply (2+4) by.
Observation: 15
BOT RESPONSE: Thought: The user provided the number 15.  Now I need to calculate (2+4) * 15.

Action: calculate: (2+4) * 15

PAUSE

 -- Running calculate (2+4) * 15
CALCULATED>>> 90
Observation: 90
BOT RESPONSE: Answer: 90



In [15]:
query("What is the population of Paris times a number plus the population of London")

BOT RESPONSE: Thought: To answer this question, I need to find the current population of Paris and London. After obtaining these figures, I can perform the calculation. I will first search for the population of Paris. 

Action: search: population of Paris 2023
PAUSE
 -- Running search population of Paris 2023
WEB SEARCH RESULTS>>> [{'title': 'Paris - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Paris', 'body': 'Between 2013 and 2023 , the population fell by 122,919, or about five percent. The Mayor of Paris , Anne Hidalgo, declared that this illustrated the "de-densification" of the city, creating more green space and less crowding.[151][152] Despite the drop, Paris remains the most densely...'}]
Observation: Between 2013 and 2023 , the population fell by 122,919, or about five percent. The Mayor of Paris , Anne Hidalgo, declared that this illustrated the "de-densification" of the city, creating more green space and less crowding.[151][152] Despite the drop, Paris remains the mos

In [52]:
from src.openai_client import generate_text

new_prompt = "What is the population of Paris times 2 plus the population of London"

generate_text(new_prompt)

'As of my last knowledge update in October 2021, the estimated population of Paris was about 2.1 million in the city proper, while the metropolitan area had a population of around 12 million. The population of London was approximately 9 million in the Greater London area.\n\nFor a rough calculation using the city proper population of Paris:\n\n1. Population of Paris (city proper) = 2.1 million\n2. Population of London = 9 million\n\nCalculation:\n\\( (2.1 \\text{ million} \\times 2) + 9 \\text{ million} = 4.2 \\text{ million} + 9 \\text{ million} = 13.2 \\text{ million} \\)\n\nPlease check the latest statistics for the most accurate figures, as populations can change.'

In [54]:
query(new_prompt)

BOT RESPONSE: Thought: To calculate the requested value, I need to find the current population of Paris and London. I can start by searching for the population of both cities. 

Action: search: population of Paris 2023 
PAUSE
' -- Running search population of Paris 2023'
Observation: Nov 28, 2025 · This graph shows the estimated population in the city of Paris from 1989 to 2025. It appears that the number of inhabitants in the French capital decreased since 2012 and from 2.24 million ...
BOT RESPONSE: Thought: I have found that the population of Paris is approximately 2.24 million. Now, I need to find the population of London to complete the calculation.

Action: search: population of London 2023 
PAUSE
' -- Running search population of London 2023'
Observation: The next planned incarnation of Londinium prospered, superseding Colchester as the principal city of the Roman province of Britannia in 100. At its height in the 2nd century, Roman London had a population of about 60,000.[41]. 

In [19]:
# Test wikisearch with a simple query (complex math questions work better with the ReAct agent above)
websearch("Paris population")

WEB SEARCH RESULTS>>> [{'title': 'Demographics of Paris - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Demographics_of_Paris', 'body': 'Learn about the population, density and growth of Paris and its regions, based on official statistics and historical data. Compare Paris with other European and global cities and see maps and charts.'}]


'Learn about the population, density and growth of Paris and its regions, based on official statistics and historical data. Compare Paris with other European and global cities and see maps and charts.'